In [ ]:
import numpy as np
import tester
from tester import run_repeated_tests, InitParamsDict, TestParamsDict
import os

In [3]:
import time
import threading
import psutil
import numpy as np

try:
    import pynvml
    pynvml.nvmlInit()
    gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    GPU_AVAILABLE = True
except:
    GPU_AVAILABLE = False

class ResourceTracker:
    def __init__(self, interval=0.1):
        self.interval = interval
        self._running = False
        self._samples = []

    def _sample(self):
        process = psutil.Process()
        while self._running:
            cpu = psutil.cpu_percent(interval=None)
            ram = process.memory_info().rss / 1e6  # MB

            gpu, vram = (0.0, 0.0)
            if GPU_AVAILABLE:
                util = pynvml.nvmlDeviceGetUtilizationRates(gpu_handle)
                mem = pynvml.nvmlDeviceGetMemoryInfo(gpu_handle)
                gpu = util.gpu
                vram = mem.used / 1e6  # MB

            self._samples.append((cpu, ram, gpu, vram))
            time.sleep(self.interval)

    def start(self):
        self._samples.clear()
        self._running = True
        self._start_time = time.time()
        self._thread = threading.Thread(target=self._sample)
        self._thread.start()

    def stop(self):
        self._running = False
        self._thread.join()
        duration = time.time() - self._start_time

        if self._samples:
            arr = np.array(self._samples)
            avg_cpu, avg_ram, avg_gpu, avg_vram = arr.mean(axis=0)
        else:
            avg_cpu = avg_ram = avg_gpu = avg_vram = 0.0

        return {
            'avg_cpu_percent': avg_cpu,
            'avg_ram_mb': avg_ram,
            'avg_gpu_percent': avg_gpu,
            'avg_vram_mb': avg_vram,
            'duration_sec': duration
        }

In [ ]:
save_path = 'resource_tests'
stats_path = os.path.join(save_path, 'stats.txt')
#tester.set_device('cuda:0')

In [ ]:
init_params_dict : InitParamsDict = {
    'test_name': 'resource_test_mnist_sp', # Changed name slightly

    'dataset_name': 'mnist',
    'num_clients': 5,
    'num_classes': 10,                # Number of classes in the dataset
    'distribution_type': 'random',     # Distribution type

    'model_name': 'simple_cnn',       # Model architecture
    'loss_name': 'cross_entropy',     # Loss function

    'trainer_name': 'sgd',            # Trainer type
    'train_epochs': 3,                # Initial training epochs

    'target_client': 0,               # Client to unlearn
    'num_tests': 1                   # Number of independent repetitions
}

test_params_dict_0 : TestParamsDict = {
        'subtest': 0,
        'unlearning_method': 'information',
        'tests': ['test_accuracy', 'clients_accuracies', 'mia'],
        'mia_classifier_types': ['nn', 'logistic'],
        'retrain_epochs': 1
    }

percentages = np.arange(5, 20, 5)
test_params_dicts_0 = [test_params_dict_0.copy() for _ in range(len(percentages))]
for i, percentage in enumerate(percentages):
    test_params_dicts_0[i]['unlearning_percentage'] = percentage

test_params_dicts = test_params_dicts_0

tracker = ResourceTracker()
tracker.start()
run_repeated_tests(init_params_dict, test_params_dicts, save_path, num_workers=1)
stats=tracker.stop()

with open(stats_path, "a") as f:
    f.write(f"\n=== Run {init_params_dict['test_name']} ===\n")
    for key, value in stats.items():
        f.write(f"{key}: {value}\n")

print(f"\n=== Run {init_params_dict['test_name']} ===")
for key, value in stats.items():
    print(f"{key}: {value}")

2025-05-09 15:31:24,569 - INFO - Starting test suite: resource_test_mnist_sp
2025-05-09 15:31:24,570 - WARNING - Test directory 'resource_tests\resource_test_mnist_sp' already exists.
2025-05-09 15:31:24,572 - INFO - Created test suite directory: resource_tests\resource_test_mnist_sp (1)
2025-05-09 15:31:24,573 - INFO - Initial Configuration:
2025-05-09 15:31:24,575 - INFO -   test_name: resource_test_mnist_sp
2025-05-09 15:31:24,577 - INFO -   dataset_name: mnist
2025-05-09 15:31:24,577 - INFO -   num_clients: 5
2025-05-09 15:31:24,578 - INFO -   num_classes: 10
2025-05-09 15:31:24,579 - INFO -   distribution_type: random
2025-05-09 15:31:24,580 - INFO -   model_name: simple_cnn
2025-05-09 15:31:24,581 - INFO -   loss_name: cross_entropy
2025-05-09 15:31:24,582 - INFO -   trainer_name: sgd
2025-05-09 15:31:24,583 - INFO -   train_epochs: 1
2025-05-09 15:31:24,584 - INFO -   target_client: 0
2025-05-09 15:31:24,585 - INFO -   num_tests: 1
2025-05-09 15:31:24,586 - INFO - --------------

Running repeated tests:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-09 15:31:24,669 - INFO - --- Starting Test Iteration 0 ---
2025-05-09 15:31:24,669 - INFO - Using device: cuda
2025-05-09 15:31:24,677 - INFO - Training model...


Training:   0%|          | 0/1 [00:00<?, ?epoch/s]

2025-05-09 15:31:43,173 - INFO - Epoch 1/1, Loss: 0.014402699656784534
2025-05-09 15:31:43,187 - INFO - Training benchmark model...


...

Computing accuracy:   0%|          | 0/375 [00:00<?, ?batch/s]

Computing accuracy:   0%|          | 0/375 [00:00<?, ?batch/s]

Computing accuracy:   0%|          | 0/375 [00:00<?, ?batch/s]

2025-05-09 15:34:45,720 - INFO - Running MIA...
2025-05-09 15:34:45,722 - INFO - Classifier type: nn
2025-05-09 15:35:07,968 - INFO - Classifier type: logistic
2025-05-09 15:35:19,537 - INFO - --- Finished Test Iteration 0 ---
2025-05-09 15:35:19,541 - INFO - Test suite 'resource_test_mnist_sp' completed


Resource usage during tests: {'avg_cpu_percent': np.float64(30.229271465741633), 'avg_ram_mb': np.float64(1400.6247470181781), 'avg_gpu_percent': np.float64(41.89549002601908), 'avg_vram_mb': np.float64(460.6026957155364), 'duration_sec': 234.99483275413513}


In [ ]:
init_params_dict : InitParamsDict = {
    'test_name': 'resource_test_mnist_mp', # Changed name slightly

    'dataset_name': 'mnist',
    'num_clients': 5,
    'num_classes': 10,                # Number of classes in the dataset
    'distribution_type': 'random',     # Distribution type

    'model_name': 'simple_cnn',       # Model architecture
    'loss_name': 'cross_entropy',     # Loss function

    'trainer_name': 'sgd',            # Trainer type
    'train_epochs': 3,                # Initial training epochs

    'target_client': 0,               # Client to unlearn
    'num_tests': 3                   # Number of independent repetitions
}

test_params_dict_0 : TestParamsDict = {
        'subtest': 0,
        'unlearning_method': 'information',
        'tests': ['test_accuracy', 'clients_accuracies', 'mia'],
        'mia_classifier_types': ['nn', 'logistic'],
        'retrain_epochs': 1
    }

percentages = np.arange(5, 20, 5)
test_params_dicts_0 = [test_params_dict_0.copy() for _ in range(len(percentages))]
for i, percentage in enumerate(percentages):
    test_params_dicts_0[i]['unlearning_percentage'] = percentage

test_params_dicts = test_params_dicts_0

tracker = ResourceTracker()
tracker.start()
run_repeated_tests(init_params_dict, test_params_dicts, save_path, num_workers=3)
stats=tracker.stop()

with open(stats_path, "a") as f:
    f.write(f"\n=== Run {init_params_dict['test_name']} ===\n")
    for key, value in stats.items():
        f.write(f"{key}: {value}\n")

print(f"\n=== Run {init_params_dict['test_name']} ===")
for key, value in stats.items():
    print(f"{key}: {value}")

In [ ]:
init_params_dict : InitParamsDict = {
    'test_name': 'resource_test_cifar_sp', # Changed name slightly

    'dataset_name': 'cifar10',
    'num_clients': 5,
    'num_classes': 10,                # Number of classes in the dataset
    'distribution_type': 'random',     # Distribution type

    'model_name': 'resnet18',       # Model architecture
    'loss_name': 'cross_entropy',     # Loss function

    'trainer_name': 'sgd',            # Trainer type
    'train_epochs': 3,                # Initial training epochs

    'target_client': 0,               # Client to unlearn
    'num_tests': 1                   # Number of independent repetitions
}

test_params_dict_0 : TestParamsDict = {
        'subtest': 0,
        'unlearning_method': 'information',
        'tests': ['test_accuracy', 'clients_accuracies', 'mia'],
        'mia_classifier_types': ['nn', 'logistic'],
        'retrain_epochs': 1
    }


percentages = np.arange(5, 20, 5)
test_params_dicts_0 = [test_params_dict_0.copy() for _ in range(len(percentages))]
for i, percentage in enumerate(percentages):
    test_params_dicts_0[i]['unlearning_percentage'] = percentage

test_params_dicts = test_params_dicts_0

tracker = ResourceTracker()
tracker.start()
run_repeated_tests(init_params_dict, test_params_dicts, save_path, num_workers=1)
stats=tracker.stop()

with open(stats_path, "a") as f:
    f.write(f"\n=== Run {init_params_dict['test_name']} ===\n")
    for key, value in stats.items():
        f.write(f"{key}: {value}\n")

print(f"\n=== Run {init_params_dict['test_name']} ===")
for key, value in stats.items():
    print(f"{key}: {value}")

In [ ]:
init_params_dict : InitParamsDict = {
    'test_name': 'resource_test_cifar_mp', # Changed name slightly

    'dataset_name': 'cifar10',
    'num_clients': 5,
    'num_classes': 10,                # Number of classes in the dataset
    'distribution_type': 'random',     # Distribution type

    'model_name': 'resnet18',       # Model architecture
    'loss_name': 'cross_entropy',     # Loss function

    'trainer_name': 'sgd',            # Trainer type
    'train_epochs': 3,                # Initial training epochs

    'target_client': 0,               # Client to unlearn
    'num_tests': 3                   # Number of independent repetitions
}

test_params_dict_0 : TestParamsDict = {
        'subtest': 0,
        'unlearning_method': 'information',
        'tests': ['test_accuracy', 'clients_accuracies', 'mia'],
        'mia_classifier_types': ['nn', 'logistic'],
        'retrain_epochs': 1
    }


percentages = np.arange(5, 20, 5)
test_params_dicts_0 = [test_params_dict_0.copy() for _ in range(len(percentages))]
for i, percentage in enumerate(percentages):
    test_params_dicts_0[i]['unlearning_percentage'] = percentage

test_params_dicts = test_params_dicts_0

tracker = ResourceTracker()
tracker.start()
run_repeated_tests(init_params_dict, test_params_dicts, save_path, num_workers=3)
stats=tracker.stop()

with open(stats_path, "a") as f:
    f.write(f"\n=== Run {init_params_dict['test_name']} ===\n")
    for key, value in stats.items():
        f.write(f"{key}: {value}\n")

print(f"\n=== Run {init_params_dict['test_name']} ===")
for key, value in stats.items():
    print(f"{key}: {value}")

2025-05-09 15:41:46,339 - INFO - Starting test suite: resource_test_cifar_mp
2025-05-09 15:41:46,343 - INFO - Created test suite directory: resource_tests\resource_test_cifar_mp
2025-05-09 15:41:46,343 - INFO - Initial Configuration:
2025-05-09 15:41:46,348 - INFO -   test_name: resource_test_cifar_mp
2025-05-09 15:41:46,349 - INFO -   dataset_name: cifar10
2025-05-09 15:41:46,350 - INFO -   num_clients: 5
2025-05-09 15:41:46,351 - INFO -   num_classes: 10
2025-05-09 15:41:46,352 - INFO -   distribution_type: random
2025-05-09 15:41:46,354 - INFO -   model_name: resnet18
2025-05-09 15:41:46,354 - INFO -   loss_name: cross_entropy
2025-05-09 15:41:46,355 - INFO -   trainer_name: sgd
2025-05-09 15:41:46,356 - INFO -   train_epochs: 3
2025-05-09 15:41:46,357 - INFO -   target_client: 0
2025-05-09 15:41:46,359 - INFO -   num_tests: 10
2025-05-09 15:41:46,360 - INFO - ------------------------------


Files already downloaded and verified
Files already downloaded and verified


2025-05-09 15:41:48,699 - INFO - Using 10 workers for parallel processing.
2025-05-09 15:41:48,699 - INFO - No devices provided, using default device cuda for all workers.
